In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('test_scores.csv')

In [3]:
data.describe()

,n_student,pretest,posttest
count,2133.000000,2133.000000,2133.000000
mean,22.796531,54.955931,67.102203
std,4.228893,13.563101,13.986789
min,14.000000,22.000000,32.000000
25%,20.000000,44.000000,56.000000
50%,22.000000,56.000000,68.000000
75%,27.000000,65.000000,77.000000
max,31.000000,93.000000,100.000000


In [4]:
data.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,student_id,gender,lunch,pretest,posttest
0,ANKYI,Urban,Non-public,6OL,Standard,20.0,2FHT3,Female,Does not qualify,62.0,72.0
1,ANKYI,Urban,Non-public,6OL,Standard,20.0,3JIVH,Female,Does not qualify,66.0,79.0
2,ANKYI,Urban,Non-public,6OL,Standard,20.0,3XOWE,Male,Does not qualify,64.0,76.0
3,ANKYI,Urban,Non-public,6OL,Standard,20.0,556O0,Female,Does not qualify,61.0,77.0
4,ANKYI,Urban,Non-public,6OL,Standard,20.0,74LOE,Male,Does not qualify,64.0,76.0


In [5]:
features = ['school', 'school_setting', 'school_type', 'classroom', 'teaching_method', 'n_student', 'gender', 'pretest']
X = data[features]
y = data.posttest

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

In [6]:
X.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,gender,pretest
0,ANKYI,Urban,Non-public,6OL,Standard,20.0,Female,62.0
1,ANKYI,Urban,Non-public,6OL,Standard,20.0,Female,66.0
2,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,64.0
3,ANKYI,Urban,Non-public,6OL,Standard,20.0,Female,61.0
4,ANKYI,Urban,Non-public,6OL,Standard,20.0,Male,64.0


In [7]:
newData = data.groupby('gender').gender.count()
print(newData.head())

gender
Female    1056
Male      1077
Name: gender, dtype: int64


## school_setting = ['Rural', 'Suburban', 'Urban']
## school_type = ['Non-public', 'Public']
## school = ['ANKYI', 'CCAAW', 'CIMBB', 'CUQAM', 'DNQDD']
## classroom = ['05H', '08N', '0N7', '18K', 197']
## teaching_method = ['experimental', 'standard']
## gender = ['Female', 'Male']


In [8]:
s = (train_X.dtypes=='object')
object_cols = list(s[s].index)

In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_col_train = pd.DataFrame(enc.fit_transform(train_X[object_cols]))
oh_col_valid = pd.DataFrame(enc.transform(val_X[object_cols]))

oh_col_train.index = train_X.index
oh_col_valid.index = val_X.index

num_X_train = train_X.drop(object_cols, axis=1)
num_X_valid = val_X.drop(object_cols, axis=1)

X_train = pd.concat([num_X_train, oh_col_train], axis=1)
X_valid = pd.concat([num_X_valid, oh_col_valid], axis=1)

In [10]:
from sklearn.tree import DecisionTreeRegressor
model_1 = DecisionTreeRegressor(max_leaf_nodes=100, random_state=0)
model_1.fit(X_train, train_y)

DecisionTreeRegressor(max_leaf_nodes=100, random_state=0)

In [11]:
from sklearn.metrics import mean_absolute_error
preds1 = model_1.predict(X_valid)
mae1 = mean_absolute_error(val_y, preds1)
print(mae1)

2.7099678801058693


# RandomForest

In [12]:
from sklearn.ensemble import RandomForestRegressor
model_2 = RandomForestRegressor(n_estimators=500, random_state=0)
model_2.fit(X_train, train_y)
preds2 = model_2.predict(X_valid)
mae2 = mean_absolute_error(val_y, preds2)
print(mae2)


2.6819016789182513


In [13]:
X_valid.head(1)

,n_student,pretest,0,1,2,3,4,5,6,7,...,119,120,121,122,123,124,125,126,127,128
1968,24.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [14]:
val_y.head(1)

1968    79.0
Name: posttest, dtype: float64

In [15]:
model_1.predict(X_valid.head(1))

array([79.43333333])

In [16]:
model_2.predict(X_valid.head(1))

array([80.1066381])